In [1]:
%cd ../../
%load_ext autoreload
%autoreload 2

/media/olle/DATA/Projects/orderbookrl


In [2]:
from orderbookrl.evaluate.collect_data import load_env_agent, run_through_all_data
import pandas as pd
%matplotlib inline

In [ ]:
agent_id = 'PPO'
path = 'logs/marketorderenvfunds/ppo/PPOADV_MarketOrderEnvFunds-v0_0_2018-09-03_19-48-52k7iu9uxw/'
checkpoint = 90
data_dir = '/media/olle/DATA/Projects/orderbookrl/data/test/'
env, agent = load_env_agent(agent_id, path, checkpoint, data_path=data_dir)
result, trades, states, actions, rewards, quotes = run_through_all_data(env, agent)

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:44569 to respond...


Using custom preprocessor mv


Waiting for redis server at 127.0.0.1:52825 to respond...
Starting local scheduler with the following resources: {'CPU': 8, 'GPU': 1}.

View the web UI at http://localhost:8889/notebooks/ray_ui99304.ipynb?token=6ca15a7cf8f13c2daabe8bce9fd7cf76ef13d2547e670f36



Created LogSyncer for /home/olle/ray_results/2018-09-03_21-14-57ufr4axzd -> None
Using custom preprocessor mv


/home/olle/miniconda3/envs/orderbookmdp/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning:

`nthreads` argument is deprecated, pass `use_threads` instead

/home/olle/miniconda3/envs/orderbookmdp/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning:

`nthreads` argument is deprecated, pass `use_threads` instead



2018-04-04T17:29:01.296000Z 0.8733536819207135
2018-04-04T20:18:01.320000Z 0.7819473218299058
2018-04-04T23:33:22.103000Z 0.6560340832020379
2018-04-05T01:47:21.887000Z 0.5834267033341651
2018-04-05T04:03:06.718000Z 0.5822612088708796
2018-04-05T06:57:26.524000Z 0.5412520080543125
2018-04-05T09:02:20.546000Z 0.5334562729276641
2018-04-05T12:18:06.871000Z 0.4689099242868097
2018-04-05T14:17:07.433000Z 0.44811471023781696
2018-04-05T15:58:06.528000Z 0.40686334236438276
2018-04-05T17:42:54.399000Z 0.37518101565183914
2018-04-05T19:28:12.853000Z 0.3378171545645032
2018-04-05T22:33:31.865000Z 0.3124734248849108
2018-04-06T01:48:26.296000Z 0.29280861296127025
2018-04-06T05:33:37.247000Z 0.2806351698134763
2018-04-06T08:58:52.350000Z 0.2600203594927676
2018-04-06T12:59:54.806000Z 0.24595411649108204
2018-04-06T17:25:06.363000Z 0.2307514211763326
2018-04-06T23:10:18.184000Z 0.22295454882364366
2018-04-07T05:07:17.008000Z 0.22913491646467937
2018-04-07T11:32:56.030000Z 0.22225671046158363
2018-

In [ ]:
actions.columns = ['BUY=0_SELL=1_HOLD=2']
result.join(actions).plot(subplots=True, figsize=(15,10));

In [ ]:
n_cols = 3
layout = (n_cols, 1+ int( (1+states.shape[1])/n_cols))
states.columns = ['ofi', 'vol_mb', 'macd', 'mid_std', 'possession']
states.plot(subplots=True, figsize=(15,10), layout=layout);

In [ ]:
mid = (quotes['ask'] + quotes['bid'])/2
mid.name = 'mid'
temp = mid.to_frame().join(result.possession)
temp = temp.join(result.capital/result.capital[0])
temp.plot(subplots=True, figsize=(10,10));

In [ ]:
port_returns = result.capital.pct_change().iloc[1:]
port_cum_ret = (1+ port_returns).cumprod()
port_cum_ret.name = 'portfolio'
cum_returns = (1 + mid.pct_change().iloc[1:]).cumprod()
cum_returns.name = 'bitcoin'
temp = pd.concat([port_cum_ret, cum_returns], axis=1)
temp.plot(figsize=(10,10));

In [ ]:
temp = mid.to_frame().join(result.possession)
temp.plot(secondary_y='possession', figsize=(15, 5));

In [ ]:
port_returns.rolling(100).mean().nlargest(n=25)

In [ ]:
temp['2018-05-13 09'].plot(subplots=True, figsize=(10,10));

In [ ]:
actions.head()

In [ ]:
states.head()

In [ ]:
states.shape

In [ ]:
from sklearn import tree
import graphviz 
import numpy as np
np.random.seed(1)
n = 10000
idx = np.random.choice(np.arange(states.shape[0]), n)
reg = tree.DecisionTreeClassifier().fit(states.iloc[idx], actions.values[idx].ravel())
dot_data = tree.export_graphviz(reg, max_depth=3, out_file=None,
                         feature_names=states.columns, class_names=['Buy', 'Sell', 'Hold'],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graphviz.Source(dot_data)  